<a href="https://colab.research.google.com/github/helmana/DLeeg/blob/master/channel_shift.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mne

In [4]:

import matplotlib

from matplotlib import pyplot as plt
import numpy as np



import random
import mne
from mne.preprocessing import create_ecg_epochs, create_eog_epochs
from mne import io
from mne import viz
#from mne.datasets import testing
from mne import Epochs, io, pick_types
from mne.event import define_target_events
from mne.time_frequency import psd_welch
print(__doc__)

Automatically created module for IPython interactive environment


In [0]:
#with np.errstate(divide='ignore'):
   # np.float64(1.0) / 0.0

In [5]:
subject_number=5


# load dataset in array
list_raw_fnames = [[0]*2]*subject_number
for x in range(subject_number):
    list_raw_fnames[x] = mne.datasets.eegbci.load_data(x+1,[1,2])

list_rawdata = np.zeros((subject_number,2), dtype='object')

for i in range(subject_number):
    for j in range(2):
        list_rawdata[i][j] = mne.io.read_raw_edf(list_raw_fnames[i][j], preload=True)
print( list_rawdata[:][:])

Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R01.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['T0']
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S001/S001R02.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 9759  =      0.000 ...    60.994 secs...
Used Annotations descriptions: ['T0']
Extracting EDF parameters from /root/mne_data/MNE-eegbci-data/physiobank/database/eegmmidb/S002/S002R01.edf...
EDF file detected
EDF annotations detected (consider using raw.find_edf_events() to extract them)
Setting channel info structure...
Creating raw.info struc

In [8]:
import math


task_number=1
task_time = 60
sampel_number_per_sec =  160 # sampel rate
total_sampel_number =  sampel_number_per_sec *task_time # 60*160
sample_shift = 4 #step len
window_len= 20

# img for each one
subject_img_number = math.floor((total_sampel_number - sampel_number_per_sec) / sample_shift) +1 - window_len +1
test_start_index=subject_img_number - math.floor(subject_img_number/4) 

train_number =  test_start_index 
test_number = subject_img_number - test_start_index
train_number

1757

In [30]:
ch_number = 2

train_img = np.zeros((train_number*subject_number, ch_number, window_len + 1, 160), dtype = float)
train_label =[]


test_img = np.zeros((test_number*subject_number, ch_number, window_len, 160), dtype = float)
test_label =[]


train_img.shape

(8785, 2, 21, 160)

In [0]:

def normalize_channel_data(ch , i, ch_min, ch_max):
  ch = ((ch - ch_min[i]) / (ch_max[i] - ch_min[i] ) )
  return ch

In [0]:
for s in range(subject_number):
    rawdataChannels_t, times_t=rawdataChannels, times =list_rawdata[s][0][:64,:9600]
    ch_max =[]
    ch_min =[]
    for p in range(len(rawdataChannels_t)):
        ch_max = np.append(ch_max, max(rawdataChannels_t[p])) # max for each cannel
        ch_min = np.append(ch_min, min(rawdataChannels_t[p])) # min for each cannel

    for j in range (subject_img_number):
        for i in range(ch_number):
            for z in range (window_len):
                rawdataChannels, times=rawdataChannels, times =list_rawdata[s][0][i,0+(j+z)*sample_shift:sampel_number_per_sec+(j+z)*sample_shift]

                      # normalize method
                rawdataChannels = normalize_channel_data(rawdataChannels, i, ch_min, ch_max)
                if j <test_start_index :
                    train_img[s*train_number + j][i][z] = rawdataChannels
                    if z==window_len-1 :
                      train_img[s*train_number + j][i][z+1] = s
                      
                      
                else:
                    test_img[s*test_number + j - test_start_index][i][z] = rawdataChannels
        if j >=test_start_index :
          test_label = np.append(test_label, (s))

           

In [0]:
#shaffle train img array
train_img_shuffle = train_img

np.random.shuffle(train_img_shuffle)

In [0]:
train_img_2 = np.zeros((train_number*subject_number, ch_number, window_len, 160), dtype = float)


In [0]:
# img label 

train_img_shuffle_len =len(train_img_shuffle)
for i in range(train_img_shuffle_len):
    train_label = np.append(train_label, (train_img_shuffle[i][0][window_len][0] ))
    for j in range (ch_number):
      train_img_2[i][j] = np.delete(train_img_shuffle[i][j], window_len, axis=0)

In [0]:
import keras
from keras.utils import to_categorical

In [0]:
train_label = to_categorical(train_label, subject_number)
test_label = to_categorical(test_label, subject_number)

In [37]:
# valid & train
x_train =train_img_2[:5000]
y_train =train_label[:5000]

x_valid =train_img_2[5000:]
y_valid =train_label[5000:]

x_train.shape

(5000, 2, 20, 160)

In [41]:
train_label[:20]

array([[1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [42]:
from keras import layers
from keras import models
from keras import regularizers


model = models.Sequential()
model.add(layers.Conv2D(64, (5,5), activation = 'relu', input_shape = (ch_number,window_len,160), data_format= "channels_first" ))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))
print(model.output.shape)
model.add(layers.Conv2D(128, (5,5), activation = 'relu'))
print(model.output.shape)
model.add(layers.MaxPooling2D((2,2)))

model.add(layers.Flatten())
print(model.output.shape)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.5))
print(model.output.shape)
model.add(layers.Dense(subject_number, activation = 'softmax'))
print(model.output.shape)

Instructions for updating:
Colocations handled automatically by placer.
(?, 64, 16, 156)
(?, 32, 8, 156)
(?, 28, 4, 128)
(?, ?)
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
(?, 512)
(?, 5)


In [0]:

from keras import optimizers

model.compile(loss= 'categorical_crossentropy',
              optimizer= optimizers.RMSprop(lr= 1e-4),
              metrics = ['acc'])

In [44]:
history = model.fit(
        x_train,
        y_train,
        epochs = 10,
        batch_size = 10,
        validation_data = (x_valid, y_valid)
)

Instructions for updating:
Use tf.cast instead.
Train on 5000 samples, validate on 3785 samples
Epoch 1/10
5000/5000 [==============================] - 104s 21ms/step - loss: 0.8538 - acc: 0.6316 - val_loss: 0.5799 - val_acc: 0.7881
Epoch 2/10
5000/5000 [==============================] - 106s 21ms/step - loss: 0.5107 - acc: 0.8026 - val_loss: 0.4570 - val_acc: 0.8235
Epoch 3/10
5000/5000 [==============================] - 106s 21ms/step - loss: 0.4041 - acc: 0.8430 - val_loss: 0.3626 - val_acc: 0.8547
Epoch 4/10
5000/5000 [==============================] - 105s 21ms/step - loss: 0.3359 - acc: 0.8674 - val_loss: 0.3256 - val_acc: 0.8634
Epoch 5/10
5000/5000 [==============================] - 105s 21ms/step - loss: 0.2863 - acc: 0.8834 - val_loss: 0.2317 - val_acc: 0.9152
Epoch 6/10
5000/5000 [==============================] - 105s 21ms/step - loss: 0.2330 - acc: 0.9146 - val_loss: 0.1858 - val_acc: 0.9321
Epoch 7/10
5000/5000 [==============================] - 105s 21ms/step - loss: 0.1

In [45]:
model.evaluate(test_img, test_label)


2925/2925 [==============================] - 15s 5ms/step


[0.1828118338748516, 0.9282051282051282]